### Project 02: Create a GUI Notebook Program

Hannah Richard March 26th 2024

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [1]:
# imports
import tkinter as tk # import tkinter module
from tkinter import ttk # import tkk
import datetime # import module for working with dates and times
from tkinter import filedialog, messagebox # import module for files
import json
import os
import customtkinter

# creating own modules
from Snippet import Snippet
from MakeNote import MakeNote
from NoteForm import NoteForm


# create MainWindow class
class MainWindow(customtkinter.CTk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("1600+700") # set the default window size
        self.config(padx=100, pady=100)
        self.title('Notebook') #set the default window title
        customtkinter.set_appearance_mode("dark")
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.notes = [] # initialize an attribute named 'notebook' as an empty list

        # create new note button
        self.new_button = customtkinter.CTkButton(self, text = 'new note', command = self.new_note) # when clicked, it runs the new_note method
        self.new_button.grid(padx = 10, pady = 20, row = 0, sticky = 'w')
        
        # create open notebook button
        self.open_notebook_button = customtkinter.CTkButton(self, text='open notebook', command = self.open_notebook) # when clicked, it runs the open_notebook method
        self.open_notebook_button.grid(padx = 10, pady = 20, row = 2, sticky = 'w')
        
        # create save note button
        self.save_button = customtkinter.CTkButton(self, text = 'save notebook', command = self.save_notebook) # when clicked, it runs the save_notebook method
        self.save_button.grid(padx = 10, pady = 20, row = 1)

    # new_note method that creates an instance of the NoteForm class to allow the input of a new note
    def new_note(self):
        note_window = NoteForm(self, self.notebook, self.notes) # create instance of NoteForm Class
        return None

    def clear_frame(self, target_frame):
        for widgets in target_frame.winfo_children():
            widgets.destroy()
    
    def show_notes(self):
        self.clear_frame(self.frame_notes)
        self.notes = []
        for note in self.notebook:
            new_note = MakeNote(master=self.frame_notes, note_dict=note)
            self.notes.apend(new_note)
            new_note.pack(padx=10, pady=10)
            new_note.config(height=3, width=40, wraplength=200, justify=tk.LEFT)

    
    # open_notebook method creates the notebook window
    def open_notebook(self):
        '''
        This method is run when the open notebook button is clicked.
        This creates a top level window called new_window.
        In this window, it displays the objects of each note the user inputs by 
        presenting only the title of the note to the user. From here, the user can click
        on the notes to see more details on the notes.
        '''
        # edit code so that when you open notebook, it opens the working directory
    
        new_window = customtkinter.CTkToplevel(self) # create top level window
        new_window.geometry("300x300+500+500") # set dimensions
        new_window.config(padx=100, pady=100)
        new_window.title("Notes") # title of window
        # tk.Label(new_window, text = "Click on the notes to get more details").pack() # gives the user instructions
        
        filepath = filedialog.askopenfilename(initialdir = os.getcwd(),
                                              filetypes = [("JSON files", "*.json")])
        
        if self.notebook: # if there is a value in the self.notebook attribute this will run
            for note in self.notebook: # iterates through each object in the self.notebook attribute
                customtkinter.CTkButton(new_window, text = f"Title: {note.note_title} ", width = 20, height = 5, command = lambda n = note: self.display_whole_note(n)).pack(padx=10, pady=10) # when clicked, this button represents the whole note by running the display_whole_note method
        else:
            customtkinter.CTkButton(new_window, text = "No notes", command = self.display_whole_note).pack() # createsa no note button if there are no notes created

    # this method displays the specific note details when the note in the notebook is clicked
    def display_whole_note(self, note): 
        '''
        This method allows the note and all of its contents to be displayed to the user when the user clicks on a note in the notebook.
        It works by on the click of the note button, a top level window is created.
        This will present the user with the title, text, link, tags, and metadata of the note they choose to click on.
        By using the lambda n=note when we call this method, it passes the specific notebook object to this display_whole_note 
        function to make sure that the correct notebook object is being presented to the user when this method is called. 
        '''
        note_window = customtkinter.CTkToplevel(self) # create top level window
        note_window.geometry("2300+500") # set dimensions 
        note_window.title("Note Details") # set title
        
        snippet_instance = Snippet(self.notes, self.notebook, note)
        
        def copy():
            content = f"Title: {note.note_title}\nText: {note.note_text}\nLink: {note.note_link}\nTag: {note.note_tag}\nMetadata: {note.note_metadata}\nAuthor: {note.note_author}"
            self.clipboard_clear()
            self.clipboard_append(content)
            messagebox.showinfo("Copied", "copied to clipboard!")
                
        customtkinter.CTkLabel(note_window, text = f"Title: {note.note_title}", height = 10, wraplength = 500).grid(padx = 5, pady = 5, row = 0, column = 0) # presents note title to user
        customtkinter.CTkLabel(note_window, text = f"Text: {note.note_text}", width=100, wraplength = 500).grid(padx = 5, pady = 5, row = 1, column = 0) # presents note text to user
        customtkinter.CTkLabel(note_window, text = f"Link: {note.note_link}", wraplength = 500).grid(padx = 5, pady = 5, row = 2, column = 0) # presents note link to user
        customtkinter.CTkLabel(note_window, text = f"Tag: {note.note_tag}", wraplength = 500).grid(padx = 5, pady = 5, row = 3, column = 0) # presents note tags to user
        customtkinter.CTkLabel(note_window, text = f"Metadata: {note.note_metadata}", wraplength = 500).grid(padx = 5, pady = 5, row = 5, column = 0) # presents note metadata to user
        customtkinter.CTkLabel(note_window, text = f"Author: {note.note_author}", wraplength = 500).grid(padx = 5, pady = 5, row = 4, column = 0) # presents note metadata to user
        customtkinter.CTkButton(note_window, text = "Click to edit note", command = lambda n = note: snippet_instance.edit_note(n)).grid(padx = 5, pady = 5, row = 6, column = 0)
        customtkinter.CTkButton(note_window, text="Copy Snippet", command = copy).grid(padx=5, pady=5, row=6, column=2)

    # this method saves the note to your computer as a txt file
    def save_notebook(self):
        '''
        This method saves the note you have just created to your computer.
        This method occurs when the save button is clicked.
        This method does not save all notes at once, only the note created just prior to clicking this button is saved.
        '''
        # ** this file path will need to be chnaged to work on the graders computer
        # notes_dict = [note.to_dict() for note in self.notes]
        file = filedialog.asksaveasfile(initialdir = os.getcwd(),
                                          defaultextension = ".json", 
                                          filetypes = [("JSON files", "*.json"),
                                         ("all files", ".*")]) # gets the working directory where files will be saved, *** may need to be changed depending on the computer that this program is being run on         

        first_note = self.notes[0]
        note_dict = {'title':first_note.note_title, 'text':first_note.note_text, 'link':first_note.note_link, 'tag':first_note.note_tag, 'meta':first_note.note_metadata} 
        
        json_out = json.dumps(note_dict)
        file.write(json_out)
        self.notes = [] # Setting self.notes to empty allows the index to always be 0
       
            
# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop() # runs the loop and window


#### Print your three notes below

In [2]:
print("First note:\n") # print first note

print("""
Title: 
Saving Files in Python

Text:
In python, there is 3 types of files you can save. These files include text files, csv files, and json files. To save a text file, you need to open a file with the “w” argument so you can write in the file. Then you need to user the “f.write” method to write in the content you want to go in the file. To read this file, we need to open the same file that we just added text to using the “r” argument. Then we need to use the “.readlines” method to read each line. We then save these lines to a list and iterate through the list. Each iteration we need to strip part of it and then take the content and add it to a dictionary or list. Or we can print the contents. To save a csv file, first we need to import the CSV module. Then, we need to open the file and use the “w” argument to write in code. We then use the “csv.DictWriter” method and use the correct arguments. Then we use the “writeheader()” method to organize the dictionary. Lastly, we iterate through notes, and write in head writerow from list we iterate through. To read in the csv file, we need to open the csv file, but this time using the “r” argument so that we read the file and to write it. Then we need to use the “DictReader” method on the csv file. This will go through the rows of the csv file and then we can add the content in each row to a list, or print it out. To save a json file, we need to first import the JSON module. Then, once we have imported the module, we need to open the file we want and use the “w” argument. Then we use the json.dump() method which writes in all the content to the file. To read a JSON file, we need to open the JSON file and then load all the content that is in the file using json.load() method. We can store all of this load content into a list or other variable.

Link:
https://www.geeksforgeeks.org/saving-text-json-and-csv-to-a-file-in-python/

Tag:
 #savingfilesiscool
""")

print("\n\n\nSecond Note:\n") # print second note

print("""
Title:
Types of Data in Python

Text:
In python, there are several data types you may encounter and need to use when programming. These data types include text, number, sequences, mapping, sets, Booleans, and none. Text data types come in the form of strings. Numeric data types come in the form of integers, floats, and complex. Sequence types come in the form of lists, tuples, and range. Mapping types come in the form od dictionaries. Set types come in the form of set or frozenset. Boolean types come in the form of bool. Lastly, none types come in the form of NoneType.

Link:
https://www.w3schools.com/python/python_datatypes.asp

Tag:
 #datatypes
""")

print("\n\n\nThird Note:\n") # print third note

print(""" 
Title:
Functions in Python

Text:
When you are coding and you are noticing that part of your code is constantly being repeated, this is a sign that you could need to use a function to simplify your code. Functions in python can be run whenever you call the function, and you can pass data through them in the form of parameters. To create a function, you use the word “def” followed by the name of your function and parenthesis. Then you use a colon and start writing your lines of code indented beneath the line you have just written. Functions can be made for anything, and they are fun to make.

Link:
https://www.w3schools.com/python/python_functions.asp

Tag:
 #def
""")

First note:


Title: 
Saving Files in Python

Text:
In python, there is 3 types of files you can save. These files include text files, csv files, and json files. To save a text file, you need to open a file with the “w” argument so you can write in the file. Then you need to user the “f.write” method to write in the content you want to go in the file. To read this file, we need to open the same file that we just added text to using the “r” argument. Then we need to use the “.readlines” method to read each line. We then save these lines to a list and iterate through the list. Each iteration we need to strip part of it and then take the content and add it to a dictionary or list. Or we can print the contents. To save a csv file, first we need to import the CSV module. Then, we need to open the file and use the “w” argument to write in code. We then use the “csv.DictWriter” method and use the correct arguments. Then we use the “writeheader()” method to organize the dictionary. Lastly, we 